## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME', 'APPLICATION_TYPE', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], inplace=True)
application_df


,AFFILIATION,ASK_AMT,IS_SUCCESSFUL
0,Independent,5000,1
1,Independent,108590,1
2,CompanySponsored,5000,0
3,CompanySponsored,6692,1
4,Independent,142590,1
...,...,...,...
34294,Independent,5000,0
34295,CompanySponsored,5000,0
34296,CompanySponsored,5000,0
34297,Independent,5000,1


In [3]:
# Bin the ASK_AMT column
bins = [0, 5000, 10000, 50000, 100000, 1000000, np.inf]
labels = ['<5k', '5k-10k', '10k-50k', '50k-100k', '100k-1M', '>1M']
application_df['ASK_AMT'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels)


In [4]:
# Convert categorical data to numeric with `pd.get_dummies`
# Generate the dummies for each categorical column
application_df = pd.get_dummies(application_df, columns=['AFFILIATION', 'ASK_AMT'])
application_df.head()

,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,ASK_AMT_<5k,ASK_AMT_5k-10k,ASK_AMT_10k-50k,ASK_AMT_50k-100k,ASK_AMT_100k-1M,ASK_AMT_>1M
0,1,0,0,1,0,0,0,1,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,1,0,0,0,0,0
3,1,1,0,0,0,0,0,0,1,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,0,1,0


In [5]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)



/var/folders/kh/xdg24th513g9kjvzg1lqylsc0000gn/T/ipykernel_84485/2823290356.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL'],1).values


In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 20
hidden_nodes_layer4 = 10
hidden_nodes_layer5 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Metal device set to: Apple M2
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1040      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 20)                620       
                                                                 
 dense_3 (Dense)             (None, 10)                210       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,421
Traina

In [8]:
from tensorflow.keras.optimizers import Adadelta

optimizer = Adadelta(learning_rate=1.0)


# Compile the model
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])


In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100


2023-05-20 19:43:39.017783: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


804/804 [==============================] - 6s 5ms/step - loss: 0.6201 - accuracy: 0.6879
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6167 - accuracy: 0.6892
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6163 - accuracy: 0.6897
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6160 - accuracy: 0.6895
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6162 - accuracy: 0.6896
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6158 - accuracy: 0.6894
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6157 - accuracy: 0.6896
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6156 - accuracy: 0.6896
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6153 - accuracy: 0.6898
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6155 - accuracy: 0.6895
E

804/804 [==============================] - 4s 5ms/step - loss: 0.6144 - accuracy: 0.6900
Epoch 82/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6145 - accuracy: 0.6901
Epoch 83/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6146 - accuracy: 0.6901
Epoch 84/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6144 - accuracy: 0.6900
Epoch 85/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6146 - accuracy: 0.6900
Epoch 86/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6145 - accuracy: 0.6901
Epoch 87/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6146 - accuracy: 0.6901
Epoch 88/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6145 - accuracy: 0.6901
Epoch 89/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6145 - accuracy: 0.6901
Epoch 90/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6143 - accuracy: 

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6186 - accuracy: 0.6837 - 948ms/epoch - 4ms/step
Loss: 0.6186091899871826, Accuracy: 0.6837317943572998


In [ ]:
# Save the model
nn.save("AlphabetSoupCharity.h5")
